In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Vector Search 快速入門指南

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/embeddings/vector-search-quickstart.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br>在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/embeddings/vector-search-quickstart.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/embeddings/vector-search-quickstart.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Smitha Venkat](https://github.com/smitha-google), [Kaz Sato](https://github.com/kazunori279)|


### 先決條件

此教學需要一個連結到帳單帳戶的 Google Cloud 專案。如果要建立新專案，請參閱 [此文件](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) 來建立專案和為專案設定帳單帳戶。


### 選擇執行階段環境

本教學課程可以在 Google Colab 或 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench) 上執行。

- 要使用 Colab：按一下 [此連結](https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/embeddings/vector-search-quickstart.zh.ipynb) 以在 Colab 中開啟教學課程。

- 要使用 Workbench： 如果這是你第一次在 Google Cloud 專案中使用 Workbench，請開啟 [Workbench 主控台](https://console.cloud.google.com/vertex-ai/workbench)，然後按一下「啟用」按鈕以啟用 Notebooks API。接著按一下 [此連結](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/embeddings/vector-search-quickstart.zh.ipynb)，然後選取現有筆記型電腦或建立新的筆記型電腦。


### 這需要多少成本？

完成本教學課程，大約需要幾美元。我們在本次教學課程中使用的 Cloud 服務定價，請參閱下列網頁：

- [Vertex AI 文字嵌入服務](https://cloud.google.com/vertex-ai/pricing#generative_ai_models)
- [Vertex AI 向量搜尋](https://cloud.google.com/vertex-ai/pricing#matchingengine)
- [Cloud Storage](https://cloud.google.com/storage/pricing)
- [Vertex AI Workbench](https://cloud.google.com/vertex-ai/pricing#notebooks) 如果你要使用的話

你可以使用 [定價計算器](https://cloud.google.com/products/calculator) 根據預期的使用狀況產生成本估計。

### **警告：教學課程後刪除物件** 

如果你使用自己的 Cloud 專案，請務必在完成本教學課程後刪除所有索引、索引端點，以及 Cloud Storage 儲存空間 (如果你使用的話還要刪除 Workbench 實例)。否則，剩餘的資源將會產生意外的費用。


## 設定

在開始使用 Vertex AI 服務之前，我們需要設定下列事項。

* 安裝 Python SDK
* 環境變數
* 驗證 (僅限 Colab) 
* 啟用 API
* 設定 IAM 權限


### 安裝 Vertex AI SDK

Vertex AI 和 Google Cloud Storage API 可透過多種方式存取，包括 REST API 和 Python SDK。在本教學課程中，我們將使用 SDK。


In [ ]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage

### 重新啟動目前的執行階段

要在此 Jupyter 執行階段中使用新安裝的套件，你必須重新啟動執行階段。你可以執行下列Cell來執行此項操作，如此將重新啟動目前的Kernel。


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ Kernel將重新啟動。請等待它完成，再繼續執行下一個步驟。⚠️</b>
</div>


### 環境變數

設定環境變數。如果需要，請將以下 `[your-project-id]` 替換成你的專案 ID 並執行。


In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [ ]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

### 驗證 (僅限 Colab) 

如果你是在 Colab 上執行這個筆記本，你需要執行以下單元格驗證。如果你使用 Vertex AI Workbench 則不需要執行此步驟，因為它已預先進行驗證。


In [ ]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 設定 IAM 權限

此外，我們還需要新增存取權限給預設服務帳戶以使用這些服務。

- 前往 Console 中的 [IAM 頁面](https://console.cloud.google.com/iam-admin/)
- 尋找預設運算服務帳戶的負責人。這應該類似於：`<project-number>-compute@developer.gserviceaccount.com`
- 按一下右方編輯按鈕，然後按一下 `加入其他角色` 以將 `Vertex AI 使用者`、`儲存管理員` 和 `服務使用管理員` 角色加入帳戶中。


### 啟用 API

執行下列操作，為 Compute Engine、Vertex AI、Cloud Storage 啟用此 Google Cloud 專案的 API。


In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com --project "{PROJECT_ID}"

## 準備範例資料

在本教學課程中，我們將使用 [TheLook 資料集](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce) 該資料集包含 [產品](bigquery-public-data.thelook_ecommerce.products) 表格，其中有約 30,000 列虛擬電子商務服裝網站的合成產品資料。

![](https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/products-table.png)

從此表格中，我們準備了 `product-embs.json` 檔案。

![](https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/product-embs-json.png)

此檔案為 JSONL 格式，且每一列的 `id` 為產品 ID，`name` 為產品名稱，`embedding` 為產品名稱在 768 維度中的內嵌，這是先前使用 [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) 所產生的。

文字內嵌代表服飾產品名稱的意義。在本教學課程中，我們將使用 Vector Search 來完成項目的 [語意搜尋](https://en.wikipedia.org/wiki/Semantic_search) 。此範例程式碼可作為其他簡單推薦系統的基礎，你可以在其中快速找到「與此商品類似的其他產品」。

若要深入瞭解如何從 BigQuery 表格資料建立內嵌並將其儲存在 JSON 檔案中，請參閱 [入門：文字內嵌 + Vertex AI Vector Search](https://github.com/doggy8088/generative-ai/blob/main/vector-search/intro-textemb-vectorsearch.ipynb)。


### 在雲端儲存空間中準備資料

要使用 Vector Search 建置索引，我們需要將內嵌檔案放置在雲端儲存空間儲存區中。以下程式碼將會：

1. 建立一個 Google Cloud Storage 儲存區。
2. 將範例檔案複製到你的雲端儲存空間儲存區。


In [ ]:
BUCKET_URI = f"gs://{PROJECT_ID}-vs-quickstart-{UID}"

In [ ]:
! gsutil mb -l "$LOCATION" -p "$PROJECT_ID" "$BUCKET_URI"
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" "$BUCKET_URI"

針對使用 Vector Search 以執行查詢，我們也需要將嵌入檔案複製到本機目錄：


In [ ]:
! gsutil cp "gs://github-repo/data/vs-quickstart/product-embs.json" . # for query tests

## 建立和部署向量搜尋索引

### 建立索引

現在可以準備將字嵌入載入 Vector Search。其 API 可在 SDK 的 [aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform) 套件內使用。


In [ ]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

使用它的 `create_tree_ah_index` 函式建立 [MatchingEngineIndex](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.MatchingEngineIndex)(Matching Engine 是 Vector Search 的先前名稱)。


In [ ]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-quickstart-index-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=10,
)

呼叫 `create_tree_ah_index` 函式後，便會開始建置索引。如果資料集很小，可能會花幾分鐘；否則，則會花費約 60 分鐘以上，具體時間取決於資料集的大小。你可以在 [向量搜尋控制台 > 索引標籤](https://console.cloud.google.com/vertex-ai/matching-engine/indexes) 查看索引建立的狀態。

#### 建立索引的參數

- `contents_delta_uri`: 儲存嵌入式 JSON 檔案的 Cloud Storage 目錄 URI
- `dimensions`: 每個嵌入式的維度大小。在這個案例中，我們使用文字嵌入式 API 的嵌入式，因此是 768。
- `approximate_neighbors_count`: 在一般情況下，我們想擷取多少個類似的項目

瀏覽 [文件](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index) 以取得有關建立索引和參數的更多詳細資料。


### 建立索引端點並部署索引

若要使用索引，你需要建立一個 [索引端點](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public)。它會像伺服器執行個體一樣，接受索引的查詢要求。


In [ ]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-quickstart-index-endpoint-{UID}", public_endpoint_enabled=True
)

本教學程式使用 [公共端點](https://cloud.google.com/vertex-ai/docs/vector-search/setup/setup#choose-endpoint)，不支援 [虛擬專用雲端 (VPC)](https://cloud.google.com/vpc/docs/private-services-access)，除非你對 VPC 有特定需求，建議你使用公共端點，儘管名稱中帶有「公共」，但這並不表示其他人可透過公共網際網路存取它，它的運作方式和其他 Vertex AI 服務的端點相同，透過 IAM 預設來保護。在建立明確的 IAM 權限之前，如我們之前所述，沒有人能夠存取端點。


使用 Index 終端點，指派唯一的已部署索引 ID，以部署 Index。


In [ ]:
DEPLOYED_INDEX_ID = f"vs_quickstart_deployed_{UID}"

In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

首次將 Index 部署到 Index Endpoint 時，將會自動建置並初始化其後端，大約需要 30 分鐘。首次部署後，將會在數秒內完成。若要查看索引部署狀態，請開啟 [向量搜尋主控台 > INDEX ENDPOINTS 標籤](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints)，然後按一下 Index Endpoint。


## 使用向量搜尋執行查詢

最後可以準備使用向量搜尋。在下列程式碼中，它會找出某個指定產品的名稱嵌入，並使用向量搜尋找出類似的產品名稱。


### 取得內嵌字來執行查詢

首先，載入內嵌字 JSON 檔案來建構一個產品名稱和內嵌字的詞典。


In [ ]:
import json

# build dicts for product names and embs
product_names = {}
product_embs = {}
with open("product-embs.json") as f:
    for l in f.readlines():
        p = json.loads(l)
        id = p["id"]
        product_names[id] = p["name"]
        product_embs[id] = p["embedding"]

透過 `product_embs` 字典，你可以指定一個產品編號來取得它的嵌入。


In [ ]:
# get the embedding for ID 6523 "cloudveil women's excursion short"
# you can also try with other IDs such as 12711, 18090, 19536 and 11863
query_emb = product_embs["6523"]

### 執行查詢

將嵌入傳遞到 `find_neighbors` 函式，以尋找相似的產品名稱。


In [ ]:
# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=[query_emb], num_neighbors=10
)

# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {product_names[neighbor.id]}")

`find_neighbors` 函式僅需幾毫秒就能取得類似項目，即使索引中有數十億項目，這要歸功於 ScaNN 演算法。向量搜尋也支援 [自動擴充](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public#autoscaling)，其能根據工作負載的需求自動調整節點數量。


# 重要：清理

如果你使用的是你自己的 Cloud 專案，而非 Qwiklab 上的暫存專案，請務必在完成本教學課程後刪除所有索引、索引端點和 Cloud Storage 儲存區。否則，剩餘的物件會**產生額外費用** 。

如果你使用的是 Workbench，你可能也需要從 [控制台](https://console.cloud.google.com/vertex-ai/workbench) 刪除筆記本。


In [ ]:
# wait for a confirmation
input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# delete Index
my_index.delete()

# delete Cloud Storage bucket
! gsutil rm -r "{BUCKET_URI}"

## 實用程式

建立或佈署索引可能需要一些時間，在這段時間中，你可能會失去與 Colab 執行階段的連線。如果你失去連線，則你可以查看 [向量搜尋主控台](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) 而不必重新建立或佈署新的索引，並使用現有的索引來繼續操作。


### 取得現有的索引

如要取得已存在的索引物件，請將下列 `[your-index-id]` 取代為索引 ID，並執行Cell。你可以在 [Vector Search Console > 索引標籤](https://console.cloud.google.com/vertex-ai/matching-engine/indexes) 查看 ID。


In [ ]:
my_index_id = "[your-index-id]"  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

### 取得現有的索引端點

如要取得現有的索引端點物件，請將下列的 `[your-index-endpoint-id]` 取代為索引端點 ID，然後執行Cell。你可以在 [Vector Search Console > INDEX ENDPOINTS 分頁](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) 上查看 ID。


In [ ]:
my_index_endpoint_id = "[your-index-endpoint-id]"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)